# Множественная проверка гипотез 

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора.     
Используя критерий знаковых рангов, необходимо провести попарное сравнение каждого классификатора с каждым. Выберать два классификатора, различие между которыми наиболее статистически значимо.

In [1]:
import pandas as pd
import numpy as np

from scipy import stats
from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 

In [2]:
df = pd.read_csv("AUCs.txt", sep='\t')

In [3]:
df.head(20)

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898
5,iris,0.936,0.931,0.916,0.931
6,liver disorders,0.661,0.668,0.609,0.685
7,lung cancer,0.583,0.583,0.563,0.625
8,lymphography,0.775,0.838,0.866,0.875
9,mushroom,1.000,1.000,1.000,1.000


In [4]:
p_data = []

for i, lhs_column in enumerate(df.columns[1:]):
    for j, rhs_column in enumerate(df.columns[1:]):
        if i >= j:
            continue
        
        _, p = stats.wilcoxon(df[lhs_column], df[rhs_column])
        p_data.append([lhs_column, rhs_column, p])

In [5]:
df_p = pd.DataFrame.from_records(p_data)
df_p.columns = ['A', 'B', 'p']

In [6]:
df_p.head(10)

,A,B,p
0,C4.5,C4.5+m,0.010757
1,C4.5,C4.5+cf,0.861262
2,C4.5,C4.5+m+cf,0.015906
3,C4.5+m,C4.5+cf,0.046333
4,C4.5+m,C4.5+m+cf,0.327826
5,C4.5+cf,C4.5+m+cf,0.022909


Cтатистически значимые, на уровне 0.05, различия, шт.: 4

Сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [7]:
reject, p_corrected, a1, a2 = multipletests(df_p.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [8]:
df_p['p_corrected'] = p_corrected
df_p['reject'] = reject

In [9]:
df_p.head(10)

,A,B,p,p_corrected,reject
0,C4.5,C4.5+m,0.010757,0.064543,False
1,C4.5,C4.5+cf,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,0.022909,0.091636,False


Ответ: 0 гипотез отвергнуто.

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [10]:
reject, p_corrected, a1, a2 = multipletests(df_p.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [11]:
df_p['p_corrected'] = p_corrected
df_p['reject'] = reject

In [12]:
df_p.head(10)

,A,B,p,p_corrected,reject
0,C4.5,C4.5+m,0.010757,0.045818,True
1,C4.5,C4.5+cf,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,0.015906,0.045818,True
3,C4.5+m,C4.5+cf,0.046333,0.069499,False
4,C4.5+m,C4.5+m+cf,0.327826,0.393391,False
5,C4.5+cf,C4.5+m+cf,0.022909,0.045818,True


Ответ: 3 гипотезы отвергнуты.   
Примечание: применение метода Бенджамини-Хохберга в этой задаче некорректно, т.к. выборки связанные.